In [1]:
import numpy as np

In [2]:
import sys
import os
import torch
import matplotlib.pyplot as plt
sys.path.append("../")

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd

In [4]:
from bengali_asr.models import Whisper
from configs.whisper_characterwise import Configs
CFG = Configs()

length of train: 934048, length of valid: 2500


In [5]:
import matplotlib.pyplot as plt

In [6]:
# lrs = []

# # Simulate a training loop
# for epoch in range(364900):
#         # Forward pass, backward pass, and optimization steps would go here
#         # ...

#         # Record current learning rate
#         lrs.append(CFG.optimizer.param_groups[0]['lr'])

#         # Step the scheduler
#         CFG.scheduler.step()

# # Plot learning rates
# plt.plot(lrs)
# plt.xlabel('Iteration')
# plt.ylabel('Learning Rate')
# plt.title('One Cycle Learning Rate Policy')
# plt.show()

In [7]:
from trainer.whisper_trainer import Trainer

In [8]:
CFG.DISTRIBUTED = False

In [9]:
trainer = Trainer(CFG)

In [11]:
z = next(iter(trainer.valid_loader))

In [16]:
trainer.model.load_state_dict(torch.load("/app/bengali-speech-recognition/workdir/whisperbase_characterlevel_wer/bestmodel_wer.pkl"))

<All keys matched successfully>

In [17]:
prediction = trainer.infer(z[0].to("cuda"))

In [28]:
truth = []
preds = []

for batch in trainer.valid_loader:
    inputs, _, target_tokens = batch

    # Initialize tokens (assuming <sos> token is 0)
    generated_tokens = trainer.infer(inputs.to(trainer.model.device)).detach().cpu()

    generated_tokens = generated_tokens[:, 1:]  # Remove the start token
    for gen,tar in zip(generated_tokens,target_tokens):
        end_pos = (gen == trainer.END_TOKEN).nonzero(as_tuple=True)[0]
        if len(end_pos) > 0:
            gen = gen[:end_pos[0]] 
        hypothesis = trainer.tokenizer.decode_torch_inference(gen)
        reference = trainer.tokenizer.decode_torch_inference(tar[tar!=-1])
        truth.append(reference)
        preds.append(hypothesis)
    break

In [30]:
preds

['তিনি বিশ্বার করে বার করেনা কার করেনা কার করেনা কারে বলে পারিতে পারেনা।',
 'তিনি বিশ্বার করে বার করেনা কার করেনা।',
 'তিনি বিশ্বার করে বার করেনা কার করে বার করেনা।',
 'তিনি কিছু কিছু কিছু করেনা।',
 'তিনি বিশ্বার করে বার করেনা কার করেনা।',
 'তিনি কিছু কিছু কিছু কিছু করেনা।',
 'তিনি বিশ্বার করে বার করেনা কার করেনা কার করেনা কারে বার করেছেন।',
 'আমার কি কিছু কিছু কিছু করে কেছে।',
 'তিনি বিশ্বার করে বার করেনা কার করেনা।',
 'তিনি কিছু কিছু কিছু কিছু করেনা।',
 'আমার কি কিছু কিছু কিছু করে কেছে।',
 'আমার কি কিছু কিছু কিছু করেছে।',
 'আমার কি কিছু কিছু কিছু করেছে।',
 'তিনি বিশ্বার করে বার করেনা কার করেনা।',
 'তিনি বিশ্বার করে বার করেনা কার করেনা।',
 'তিনি কিছু কিছু কিছু করেনা।',
 'তিনি বিশ্বার করে বার করেনা কার করেনা।',
 'তিনি বিশ্বার করে বার করেনা কার করেনা।',
 'তিনি কিছু কিছু কিছু করেনা।',
 'তিনি বিশ্বার করে বার করেনা কার করেনা।',
 'তিনি কিছু কিছু কিছু কিছু করেনা।',
 'আমার কি কিছু কিছু কিছু করেছে।',
 'তিনি বিশ্বার করে বার করেনা কার করেনা।',
 'তিনি বিশ্বার করে বার করেনা কার করেনা।',
 'তিনি বিশ

In [22]:
truth

['চতুর্থ সপ্তাহ থেকে অবস্থার উন্নতি হতে থাকে,কিন্তু এই উন্নতির হারের কিছু হেরফের আছে।',
 'তার ছেলে আইনজীবী আসিফ আনোয়ার মুরাদ।',
 'এটি ছিল তার প্রথম চলচ্চিত্র এবং এই কাজের মধ্য দিয়ে তিনি ভারত জুড়ে পরিচিতি লাভ করেন।',
 'মানচিত্র দেখা তাঁর বিশেষ কয়েকটি শখের মধ্যে একটি।',
 'এরপর সবাই স্নান করেন।',
 'শাহরুখ খান পরবর্তীতে তার ভক্তের মৃত্যুতে শোক প্রকাশ করেন।',
 'মহেশপুর ইউনিয়নের অনন্য গ্রামের শিক্ষা প্রতিষ্ঠান সমূহ',
 'জেলে থাকাকালীন আইন পড়েছিলেন তিনি।',
 'সুতরাং, মন্দির তৈরির অভিজ্ঞতা তাঁদের আগেই ছিল।',
 'কেননা দেশটিতে দারিদ্রতা ব্যাপকহারে বৃদ্ধি পাচ্ছে।',
 'তিনি মুম্বইয়ে ইংরেজি থিয়েটারের সাথে যুক্ত ছিলেন।',
 'মেঘনা সেতু জাপানের সহায়তায় নির্মিত পৃথিবীর একক বৃহত্তম প্রকল্প।',
 'তিনি চার বছর বয়স থেকেই সেতার বাজানো শুরু করেছিলেন।',
 'তার উল্লেখযোগ্য শিক্ষক ছিলে পঞ্চম দলাই লামা ও চতুর্থ পাঞ্চেন লামা।',
 'এটি পুরো কাঞ্চনজঙ্ঘা পর্বতের একটি প্রাথমিক দৃষ্টিভঙ্গি সরবরাহ করে।',
 'স্বাধীনতার পর তিনি সংবিধান রচনায় গুরুত্বপূর্ণ ভূমিকা রাখেন এবং বাংলাদেশ বার কাউন্সিলের সহ-সভাপতি ছিলেন।',
 'লি